In [12]:
import json
import pandas as pd

with open('backtest_result.json') as f:
    data = json.load(f)

# json to dataframe
df = pd.read_json(data['NOTUSDT'])

# select rows with signal = 1or-1
df = df[df['signal'] != 0]
df

/var/folders/gx/l6xk27y12x5fgvygfn0h4l240000gn/T/ipykernel_37183/348831368.py:8: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(data['NOTUSDT'])


,datetime,open,high,low,close,volume,date,can_entry,upper_band,middle_band,lower_band,ATR,signal,position,take_profit,stop_loss,daily_return,strategy_return,cumulative_market_return,cumulative_strategy_return
307,2024-05-29 09:00:00,0.009111,0.009165,0.008569,0.008602,10332866474,2024-05-29,1,0.009634,0.009166,0.008697,0.000457,1,1,0.009516,0.008145,-0.055763,0.000000,1.403492,1.000000
312,2024-05-29 14:00:00,0.009242,0.009708,0.009230,0.009604,10354530522,2024-05-29,1,0.009710,0.009175,0.008640,0.000430,-1,0,0.009516,0.008145,0.039281,0.039281,1.566977,1.116485
